# **12. 어텐션이면 충분한 막강한 트랜스포머**
---
* 출판사 : 생능 출판사( http://www.booksr.co.kr/ )
* 으뜸 파이썬 저자 : 강영민, 박동규, 김성수
*  소스코드 저장소 : https://github.com/dknife/ML2nd
*  저작권 : 본 노트북 코드는 자유롭게 배포가능하지만 위의 출판사, 저서, 저자표기와 함께 배포해 주십시오.
---


### **12장 미니 프로젝트 C1** -트랜스포머를 이용한 감정 분류


In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd
import numpy as np
import urllib.request


In [3]:
# 2. NSMC 데이터 다운로드 & 로드
url = "https://raw.githubusercontent.com/e9t/nsmc/master/"

urllib.request.urlretrieve(url + "ratings_train.txt", "train.txt")
urllib.request.urlretrieve(url + "ratings_test.txt", "test.txt")

train_df = pd.read_csv("train.txt", sep='\t', header=0,
                       names=['id', 'document', 'label'])
test_df = pd.read_csv("test.txt", sep='\t',
                       header=0, names=['id', 'document', 'label'])

train_df = train_df[['document', 'label']].dropna()
test_df = test_df[['document', 'label']].dropna()
train_df['document'] = train_df['document'].astype(str)
test_df['document'] = test_df['document'].astype(str)


훈련 데이터: 149995, 테스트 데이터: 49997


In [4]:
print(f"훈련 데이터: {len(train_df)}, 테스트 데이터: {len(test_df)}")
train_df.head()

훈련 데이터: 149995, 테스트 데이터: 49997


,document,label
0,아 더빙.. 진짜 짜증나네요 목소리,0
1,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,너무재밓었다그래서보는것을추천한다,0
3,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [5]:
# 3. 토크나이저
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=10000, oov_token="<OOV>")
tokenizer.fit_on_texts(train_df['document'])

In [6]:
def encode(texts, maxlen=80):
    seq = tokenizer.texts_to_sequences(texts)
    return pad_sequences(seq, maxlen=maxlen, padding='post')

X_train = encode(train_df['document'])
X_test = encode(test_df['document'])
y_train = train_df['label'].values
y_test = test_df['label'].values

In [7]:
# 위치 인코딩 (기존과 같은 방법)
def positional_encoding(position, d_model):
   angle_rads = np.arange(position)[:, np.newaxis] * 1 / np.power(10000,
                      (2 * (np.arange(d_model)[np.newaxis, :] // 2)) / \
                      np.float32(d_model))
   angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])
   angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])
   return tf.cast(angle_rads[np.newaxis, ...], dtype=tf.float32)

In [8]:
# 4. Keras Transformer Encoder (수정된 TransformerBlock)
class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super().__init__()
        # key_dim은 보통 embed_dim // num_heads로 설정
        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim//num_heads)
        self.ffn = keras.Sequential([
            layers.Dense(ff_dim, activation="relu"),
            layers.Dense(embed_dim)
        ])
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    # 원본 토큰 ID 텐서(inputs)를 mask_input으로 받기.
    def call(self, inputs, mask_input=None, training=False):

        attention_mask = None
        if mask_input is not None:
            # 1. 패딩 마스크 생성: mask_input은 토큰 ID 텐서 (batch, seq_len)
            # 0(패딩)인 위치는 True
            padding_mask = tf.cast(tf.math.equal(mask_input, 0), tf.float32)

            # 2. 마스킹 값을 매우 큰 음수로 설정하여 어텐션 가중치를 0으로 만듦
            attention_mask = padding_mask * -1e9

            # 3. MultiHeadAttention에 맞는 4차원 텐서로 확장: (batch, 1, 1, seq_len)
            # (batch, num_heads, Q_seq_len, K_seq_len) 중 K_seq_len에 적용
            attention_mask = attention_mask[:, tf.newaxis, tf.newaxis, :]

            # (Look-ahead mask는 디코더에서만 필요하므로, 인코더인 여기서는 패딩 마스크만 사용)

        # 어텐션 계산
        # attention_mask가 None이면 마스킹 없음
        attn_output = self.att(inputs, inputs, attention_mask=attention_mask)

        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)

        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

In [11]:
# 모델 정의
embed_dim = 128
num_heads = 8
ff_dim = 512
num_blocks = 4
maxlen = 80
vocab_size = 10000

inputs = layers.Input(shape=(maxlen,)) # **이 텐서가 원본 토큰 ID 텐서**
embedding = layers.Embedding(vocab_size, embed_dim)(inputs)
x = embedding + positional_encoding(maxlen, embed_dim)

for _ in range(num_blocks):
   x = TransformerBlock(embed_dim, num_heads, ff_dim)\
                       (x, mask_input=inputs)

x = layers.GlobalAveragePooling1D()(x)
x = layers.Dropout(0.3)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)

model = keras.Model(inputs, outputs)
model.summary()

Model: "functional_9"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_5       │ (None, 80)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_1         │ (None, 80, 128)   │  1,280,000 │ input_layer_5[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_1 (Add)         │ (None, 80, 128)   │          0 │ embedding_1[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ transformer_block_4 │ (None, 80, 128)   │    198,272 │ add_1[0][0],      │
│ (TransformerBlock)  │                   │            │ input_layer_5[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ transformer_block_5 │ (None, 80, 128)   │    198,272 │ transformer_bloc… │
│ (TransformerBlock)  │                   │            │ input_layer_5[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ transformer_block_6 │ (None, 80, 128)   │    198,272 │ transformer_bloc… │
│ (TransformerBlock)  │                   │            │ input_layer_5[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ transformer_block_7 │ (None, 80, 128)   │    198,272 │ transformer_bloc… │
│ (TransformerBlock)  │                   │            │ input_layer_5[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 128)       │          0 │ transformer_bloc… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_25          │ (None, 128)       │          0 │ global_average_p… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_17 (Dense)    │ (None, 1)         │        129 │ dropout_25[0][0]  │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 2,073,217 (7.91 MB)

 Trainable params: 2,073,217 (7.91 MB)

 Non-trainable params: 0 (0.00 B)

In [12]:
# 하이퍼파라미터 설정
model.compile(
   optimizer=keras.optimizers.Adam(3e-4),
   loss="binary_crossentropy", metrics=["accuracy"]
)

# 학습 실시
history = model.fit(
   X_train, y_train, validation_data=(X_test, y_test),
   batch_size=64, epochs=5
)

Epoch 1/5
2344/2344 ━━━━━━━━━━━━━━━━━━━━ 88s 29ms/step - accuracy: 0.5204 - loss: 0.7011 - val_accuracy: 0.7159 - val_loss: 0.5543
Epoch 2/5
2344/2344 ━━━━━━━━━━━━━━━━━━━━ 57s 24ms/step - accuracy: 0.7568 - loss: 0.5167 - val_accuracy: 0.7751 - val_loss: 0.4704
Epoch 3/5
2344/2344 ━━━━━━━━━━━━━━━━━━━━ 59s 25ms/step - accuracy: 0.7932 - loss: 0.4436 - val_accuracy: 0.7934 - val_loss: 0.4342
Epoch 4/5
2344/2344 ━━━━━━━━━━━━━━━━━━━━ 54s 23ms/step - accuracy: 0.8082 - loss: 0.4060 - val_accuracy: 0.7989 - val_loss: 0.4245
Epoch 5/5
2344/2344 ━━━━━━━━━━━━━━━━━━━━ 55s 23ms/step - accuracy: 0.8179 - loss: 0.3842 - val_accuracy: 0.7881 - val_loss: 0.4324


In [13]:
# 6. 평가

from sklearn.metrics import accuracy_score

test_pred = (model.predict(X_test) > 0.5).astype(int).flatten()
acc = accuracy_score(y_test, test_pred)
print(f"최종 정확도: {acc:.4f}")


1563/1563 ━━━━━━━━━━━━━━━━━━━━ 9s 5ms/step
최종 정확도: 0.7881


In [14]:

!pip install -q gradio

import gradio as gr

In [16]:

# 7. Gradio 데모
def predict(text):
    seq = encode([text])
    pred = model.predict(seq)[0][0]
    return f"긍정 리뷰입니다! ({pred:.1%})" if pred > 0.5 else f"부정 리뷰입니다... ({1-pred:.1%})"


In [17]:

interface = gr.Interface(
    fn=predict,
    inputs=gr.Textbox(lines=1, placeholder="여러분의 평가를 입력하세요..."),
    outputs="text",
    title="으뜸 머신러닝(개정판)-트랜스포머 인코더로 감정 분석)",
    description=f"정확도 {acc:.1%} : 텐서플로우를 이용한 트랜스포머 블록 구현",
    examples=[
        ["이 영화 진짜 명작이에요! 최고!"],
        ["돈이 너무 아까워!!"],
        ["그냥 그랬어요."]
    ]
)


interface.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://013a6099df2cec02f2.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
